In [6]:
from utils.feature_extraction import get_audio_features
import numpy as np
import wave
import pyaudio
import IPython.display as ipd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [7]:
CHUNK = 1024
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 2
RATE = 44100 #sample rate
RECORD_SECONDS = 4
WAVE_OUTPUT_FILENAME = "demo_audio.wav"
emotions = ["positive", "negative", "neutral"]

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [8]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('./model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Trained_Models/model_noise_xvector.h5")
print("Loaded model from disk")

Loaded model from disk


In [9]:
demo_audio_path = './demo_audio.wav'
ipd.Audio(demo_audio_path,rate=RATE)

In [10]:
# demo_audio_path = '.\Dataset\positive\surprise040.wav'
features_not_rowed= get_audio_features(demo_audio_path,20000)

C:\Users\lielb\PycharmProjects\NLP\utils\feature_extraction.py:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  speechbrain_X = torch.tensor([X])


In [11]:
featurers_row = []
for feature in features_not_rowed:
    featurers_row.extend(feature)

In [12]:
featurers_row = np.array(featurers_row)

In [13]:
features_cnn = np.expand_dims(featurers_row,axis=[0,2])

In [14]:
preds = loaded_model.predict(features_cnn,
                         batch_size=32,
                         verbose=1)

1/1 [==============================] - 0s 121ms/step


In [15]:
preds

array([[6.6162505e-07, 9.9999928e-01, 1.0247461e-13]], dtype=float32)

In [16]:
# emotions = ["positive", "negative", "neutral"]
index = preds.argmax(axis=1).item()
index

1

In [17]:
emotions[index]

'negative'